# Import Packages

In [3]:
import warnings
warnings.filterwarnings("ignore")

#Basic
import pandas as pd
import gensim
import nltk
import re
import numpy as np
import math

#For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#For Data Preparation
import itertools
from itertools import combinations

#For Class Definition
from sklearn.base import BaseEstimator, TransformerMixin

# Import Data

In [7]:
df = pd.read_csv('C:/Users\Louis Owen/Desktop/NLP_Stacking_Ensemble/df_prepared.csv')
df=df.drop(['Unnamed: 0'],1)

# Data Cleaning

## Basic

In [8]:
df.head(2)

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,...,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_TFIDF_2-grams,Avg_TFIDF_3-grams,Avg_TFIDF_4-grams,caps_word,hashtags
0,$NFLX,NaN,0.000000,"[3pm,24pm)",0.0,0.0,NaN,-0.494,stocktwits,['out $NFLX -.35'],...,NaN,NaN,0.0,0.0,4.275428,NaN,NaN,NaN,0,0
1,$PLUG,1.0,0.266667,"[0am,9am)",0.0,0.0,NaN,0.403,stocktwits,['Very intrigued with the technology and growt...,...,0.0,0.0,0.0,0.0,4.984655,7.322638,7.589614,7.589614,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 71 columns):
cashtag                       1454 non-null object
conversation_parent           163 non-null float64
conversation_replies          919 non-null float64
created_at                    1454 non-null object
liked_by_self                 919 non-null float64
official_account              919 non-null float64
sentiment                     162 non-null object
sentiment score               1454 non-null float64
source                        1454 non-null object
spans                         1454 non-null object
text                          1454 non-null object
total_likes                   919 non-null float64
clean_text                    1454 non-null object
base_text                     1454 non-null object
POS_VB                        1454 non-null int64
POS_VBD                       1454 non-null int64
POS_VBG                       1454 non-null int64
POS_VBN                      

## Missing Values Analysis

In [10]:
def missing_values(df):
  '''
  Function to check features with missing values
  '''
  missing_values_feat=[]
  for column in df.columns:
    if df[column].isnull().values.any():
      missing_values_feat.append(column)
  return(missing_values_feat)

missing_values(df)

['conversation_parent',
 'conversation_replies',
 'liked_by_self',
 'official_account',
 'sentiment',
 'total_likes',
 'SentiWordNet_max_score',
 'SentiWordNet_min_score',
 'Avg_TFIDF_1-grams',
 'Avg_TFIDF_2-grams',
 'Avg_TFIDF_3-grams',
 'Avg_TFIDF_4-grams']

In [11]:
len(df[df.source=='stocktwits'])

919

In [12]:
df[['conversation_parent']].describe()

,conversation_parent
count,163.000000
mean,0.723926
std,0.448431
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


Remove [conversation_parent], because too much missing values

In [13]:
df[['conversation_replies']].describe()

,conversation_replies
count,919.000000
mean,0.011643
std,0.052016
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


impute -1 for missing values, because the missing values is from twitter data

In [14]:
df[['liked_by_self']].describe()

,liked_by_self
count,919.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


Remove [liked_by_self], because the missing value is from twitter data

In [15]:
df[['official_account']].describe()

,official_account
count,919.000000
mean,0.132753
std,0.339492
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


impute -1 for missing values, because the missing value is from twitter data

In [16]:
df[['sentiment']].describe()

,sentiment
count,162
unique,2
top,Bullish
freq,123


In [17]:
df['sentiment'].unique()

array([nan, 'Bearish', 'Bullish'], dtype=object)

OHE: 1 for bullish -1 for bearish else 0, because the nan values assumed to be neutral

In [18]:
df[['total_likes']].describe()

,total_likes
count,919.000000
mean,0.046971
std,0.124470
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


impute -1 for missing values, becaue the missing data is from twitter data

In [19]:
df[['SentiWordNet_max_score']].describe()

,SentiWordNet_max_score
count,1430.000000
mean,0.232955
std,0.260000
min,-0.750000
25%,0.000000
50%,0.125000
75%,0.375000
max,1.000000


In [20]:
df[pd.isnull(df.SentiWordNet_max_score)].head()

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,...,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_TFIDF_2-grams,Avg_TFIDF_3-grams,Avg_TFIDF_4-grams,caps_word,hashtags
0,$NFLX,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.494,stocktwits,['out $NFLX -.35'],...,NaN,NaN,0.0,0.0,4.275428,NaN,NaN,NaN,0,0
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0
172,$AAPL,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.270,stocktwits,['Out $AAPL'],...,NaN,NaN,0.0,0.0,4.275428,NaN,NaN,NaN,0,0
222,$SPY,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.114,stocktwits,['cover all'],...,NaN,NaN,0.0,0.0,5.356660,7.589614,NaN,NaN,0,0
277,$NEON,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.430,stocktwits,['oversold'],...,NaN,NaN,0.0,0.0,6.673323,NaN,NaN,NaN,0,0


Impute 0 for missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and because 0 is neutral 

In [21]:
df[['SentiWordNet_min_score']].describe()

,SentiWordNet_min_score
count,1430.000000
mean,-0.188199
std,0.258757
min,-1.000000
25%,-0.375000
50%,0.000000
75%,0.000000
max,0.500000


In [22]:
df[pd.isnull(df.SentiWordNet_min_score)].head()

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,...,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_TFIDF_2-grams,Avg_TFIDF_3-grams,Avg_TFIDF_4-grams,caps_word,hashtags
0,$NFLX,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.494,stocktwits,['out $NFLX -.35'],...,NaN,NaN,0.0,0.0,4.275428,NaN,NaN,NaN,0,0
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0
172,$AAPL,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,-0.270,stocktwits,['Out $AAPL'],...,NaN,NaN,0.0,0.0,4.275428,NaN,NaN,NaN,0,0
222,$SPY,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.114,stocktwits,['cover all'],...,NaN,NaN,0.0,0.0,5.356660,7.589614,NaN,NaN,0,0
277,$NEON,NaN,0.0,"[3pm,24pm)",0.0,0.0,NaN,0.430,stocktwits,['oversold'],...,NaN,NaN,0.0,0.0,6.673323,NaN,NaN,NaN,0,0


Impute 0 for missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and because 0 is neutral 

In [23]:
df[['Avg_TFIDF_1-grams']].describe()

,Avg_TFIDF_1-grams
count,1448.000000
mean,5.306872
std,0.795715
min,1.981975
25%,4.810385
50%,5.243674
75%,5.697248
max,15.179228


In [24]:
df[pd.isnull(df['Avg_TFIDF_1-grams'])]

,cashtag,conversation_parent,conversation_replies,created_at,liked_by_self,official_account,sentiment,sentiment score,source,spans,...,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_TFIDF_2-grams,Avg_TFIDF_3-grams,Avg_TFIDF_4-grams,caps_word,hashtags
45,$FCX,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.339,stocktwits,['$FCX +3.53%'],...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0
291,$CAT,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.586,stocktwits,['$CAT +5.10%'],...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0
295,$X,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.144,stocktwits,['$X +2.27%'],...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0
761,$YHOO,NaN,0.0,"[9am,3pm)",0.0,0.0,NaN,0.160,stocktwits,['$YHOO +2.61%'],...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0
1230,$SBUX,NaN,NaN,"[9am,3pm)",NaN,NaN,NaN,-0.170,twitter,"['Deutsche Bank: cuts #Starbucks to Hold', 'pr...",...,0.375,0.375,1.0,0.0,NaN,NaN,NaN,NaN,1,1
1302,$INTU,NaN,NaN,"[3pm,24pm)",NaN,NaN,NaN,0.161,twitter,['now a cloud software company. How the compan...,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,0


Impute 0 for all tf-idf missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value

## Missing Values Imputation

Missing Value Actionable Items:
1.   Remove [conversation_parent], because too much missing values
2.   Impute -1 for [conversation_replies] missing values, because the missing values is from twitter data
3.   Remove [liked_by_self], because constant feature
4.   Impute -1 for [official_account] missing values, because the missing value is from twitter data
5.   OHE [sentiment]: 1 for bullish -1 for bearish else 0, because the nan values assumed to be neutral
6.   Impute -1 for [total_likes] missing values, becaue the missing data is from twitter data
7.   Impute 0 for [SentiWordNet_max_score] missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 
8.   Impute 0 for [SentiWordNet_min_score] missing values, because the missing value is because no such word in the sentence that belongs to the sentiwordnet synsets and becaue 0 is neutral 
9.   Impute 0 for all tf-idf missing values, because the missing value is because no such word which pass the n-gram filter and 0 is the neutral value






In [25]:
df=df.drop(['conversation_parent','liked_by_self'],1)
df['conversation_replies']=df['conversation_replies'].fillna(-1)
df['official_account']=df['official_account'].fillna(-1)
df['sentiment']=df['sentiment'].apply(lambda x: 1 if x=='Bullish' else -1 if x=='Bearish' else 0)
df['total_likes']=df['total_likes'].fillna(-1)
df['SentiWordNet_max_score']=df['SentiWordNet_max_score'].fillna(0)
df['SentiWordNet_min_score']=df['SentiWordNet_min_score'].fillna(0)
df['Avg_TFIDF_1-grams']=df['Avg_TFIDF_1-grams'].fillna(0)
df['Avg_TFIDF_2-grams']=df['Avg_TFIDF_2-grams'].fillna(0)
df['Avg_TFIDF_3-grams']=df['Avg_TFIDF_3-grams'].fillna(0)
df['Avg_TFIDF_4-grams']=df['Avg_TFIDF_4-grams'].fillna(0)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 69 columns):
cashtag                       1454 non-null object
conversation_replies          1454 non-null float64
created_at                    1454 non-null object
official_account              1454 non-null float64
sentiment                     1454 non-null int64
sentiment score               1454 non-null float64
source                        1454 non-null object
spans                         1454 non-null object
text                          1454 non-null object
total_likes                   1454 non-null float64
clean_text                    1454 non-null object
base_text                     1454 non-null object
POS_VB                        1454 non-null int64
POS_VBD                       1454 non-null int64
POS_VBG                       1454 non-null int64
POS_VBN                       1454 non-null int64
POS_VBP                       1454 non-null int64
POS_VBZ                     

## Constant Features

In [27]:
class remove_constant_features(BaseEstimator,TransformerMixin):
  '''
  Class for removing constant features either categorical or numeric features
  
  Tolerance==0.01 means the numeric features is 99% constant
  '''
  def __init__(self,tolerance,verbose=False):
    self.verbose=verbose
    self.tolerance=tolerance
  
  def fit(self,df):
    self.df=df
    self.num_constant_features=[]
    self.cat_constant_features=[]
    columns=self.df.columns.tolist()
    for column in columns:
      # check constant features for numerical columns
      if np.issubdtype(self.df[column].dtype, np.number):
        if self.df[column].std()<=self.tolerance:
          self.num_constant_features.append(column)
      else:
        try:
          # check constant features for categorical columns
          if len(self.df[feat].unique())== 1:
            self.cat_constant_features.append(column)
        except:
          None
    return(self)
  
  def transform(self,df):
    self.df=self.df.drop(self.num_constant_features,1)
    self.df=self.df.drop(self.cat_constant_features,1)
    if self.verbose:
      print('Removed Features: ',self.num_constant_features+self.cat_constant_features)
      print('')
    return(self.df)

In [28]:
constant=remove_constant_features(tolerance=0.01,verbose=True)
df=constant.fit_transform(df.copy())

Removed Features:  ['POS_VM', '$num', 'num/num/num', 'call_-num%', 'General_Inquirer_pos_ratio', 'General_Inquirer_neg_ratio']



In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 63 columns):
cashtag                   1454 non-null object
conversation_replies      1454 non-null float64
created_at                1454 non-null object
official_account          1454 non-null float64
sentiment                 1454 non-null int64
sentiment score           1454 non-null float64
source                    1454 non-null object
spans                     1454 non-null object
text                      1454 non-null object
total_likes               1454 non-null float64
clean_text                1454 non-null object
base_text                 1454 non-null object
POS_VB                    1454 non-null int64
POS_VBD                   1454 non-null int64
POS_VBG                   1454 non-null int64
POS_VBN                   1454 non-null int64
POS_VBP                   1454 non-null int64
POS_VBZ                   1454 non-null int64
+num                      1454 non-null int64
-num

## Duplicated Features

In [30]:
class remove_duplicated_features(BaseEstimator,TransformerMixin):
  '''
  Class for removing duplicated features
  '''
  def __init__(self,verbose=False):
    self.verbose=verbose
    
  def fit(self,df):
    self.df=df
    self.duplicated_feat = []
    for i in range(0, len(self.df.columns)):
      col_1 = self.df.columns[i]
      
      for col_2 in self.df.columns[i + 1:]:
        # if the features are duplicated
        if self.df[col_1].equals(self.df[col_2]):
            self.duplicated_feat.append(col_2)
    return(self)
  
  def transform(self,df):
    self.df=self.df.drop(self.duplicated_feat,1)
    if self.verbose:
      print('Removed Features: ',self.duplicated_feat)
      print('')
    return(self.df)

In [31]:
duplic=remove_duplicated_features(verbose=True)
df=duplic.fit_transform(df.copy())

Removed Features:  []



## Encoding

In [32]:
df['created_at']=df['created_at'].apply(lambda x: 0 if x=='[0am,9am)' else 1 if x=='[9am,3pm)' else 2)

## Correlated Features


In [33]:
def correlated(df,tolerance):
  '''
  Function to know correlated variables
  Input: {dataframe, correlation_tolerance}
  '''
  cate_features_index = np.where(df.dtypes == 'O')[0]
  num_features_index = [x for x in range(len(df.columns)) if x not in cate_features_index]
  
  #Check correlation of numerical variables
  corrmat = df.iloc[:,num_features_index].corr()
  corrmat = corrmat.abs().unstack() # absolute value of corr coef
  corrmat = corrmat.sort_values(ascending=False)
  corrmat = corrmat[corrmat >= tolerance]
  corrmat = corrmat[corrmat < 1]
  corrmat = pd.DataFrame(corrmat).reset_index()
  corrmat.columns = ['feature1', 'feature2', 'corr']

  # find groups of correlated features

  grouped_feature_ls = []
  correlated_groups = []

  for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:
      # find all features correlated to a single feature
      correlated_block = corrmat[corrmat.feature1 == feature]
      grouped_feature_ls = grouped_feature_ls + list(
      correlated_block.feature2.unique()) + [feature]

      # append the block of features to the list
      correlated_groups.append(correlated_block)

  print('found {} correlated groups'.format(len(correlated_groups)))
  print('out of {} total features'.format(df.iloc[:,num_features_index].shape[1]))

  for group in correlated_groups:
    print(group)
    print()

In [34]:
correlated(df.drop('source',1),0.9)

found 1 correlated groups
out of 57 total features
               feature1     feature2      corr
0  conversation_replies  total_likes  0.980199



Remove conversation_replies, num%

In [35]:
df=df.drop(['conversation_replies','num%'],1)

## Final Touch

In [36]:
df['spans']=df['spans'].apply(lambda x: x[2:-2])

# Data Report

In [37]:
df.head(2)

,cashtag,created_at,official_account,sentiment,sentiment score,source,spans,text,total_likes,clean_text,...,SentiWordNet_max_score,SentiWordNet_min_score,SentiWordNet_pos_ratio,SentiWordNet_neg_ratio,Avg_TFIDF_1-grams,Avg_TFIDF_2-grams,Avg_TFIDF_3-grams,Avg_TFIDF_4-grams,caps_word,hashtags
0,$NFLX,2,0.0,0,-0.494,stocktwits,out $NFLX -.35,out $NFLX -.35,0.0,out $NFLX -.35,...,0.0,0.0,0.0,0.0,4.275428,0.000000,0.000000,0.000000,0,0
1,$PLUG,0,0.0,0,0.403,stocktwits,Very intrigued with the technology and growth ...,Been doing some work on $PLUG this evening. Ve...,1.0,Been doing some work on $PLUG this evening Ver...,...,0.0,0.0,0.0,0.0,4.984655,7.322638,7.589614,7.589614,0,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 61 columns):
cashtag                   1454 non-null object
created_at                1454 non-null int64
official_account          1454 non-null float64
sentiment                 1454 non-null int64
sentiment score           1454 non-null float64
source                    1454 non-null object
spans                     1454 non-null object
text                      1454 non-null object
total_likes               1454 non-null float64
clean_text                1454 non-null object
base_text                 1454 non-null object
POS_VB                    1454 non-null int64
POS_VBD                   1454 non-null int64
POS_VBG                   1454 non-null int64
POS_VBN                   1454 non-null int64
POS_VBP                   1454 non-null int64
POS_VBZ                   1454 non-null int64
+num                      1454 non-null int64
-num                      1454 non-null int64
+num%  

# Data Split

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['sentiment score'],1), df['sentiment score'],stratify=df['source'],test_size=0.2)

In [40]:
X_train['sentiment score']=y_train
X_test['sentiment score']=y_test
X_train=X_train.reset_index()
X_test=X_test.reset_index()

# Export

In [0]:
X_train.to_csv('C:/Users/Louis Owen/Desktop/NLP_Stacking_Ensemble/df_train_final.csv')
X_test.to_csv('C:/Users/Louis Owen/Desktop/NLP_Stacking_Ensemble/df_test_final.csv')